In [ ]:
# Convertir la duración  a segundos 
df_final_web_data['duration_seconds'] = df_final_web_data['duration'].dt.total_seconds()

In [ ]:
# Unión de tablas final_web_data y experiment_clients, para tener la información del tipo de cliente (control / prueba) dentro del estudio
df_web_tipo_cliente = pd.merge(df_final_web_data, df_final_experiment_clients[['client_id', 'Variation']], on='client_id', how='inner')

In [ ]:
# Agrupar y calcular la Mediana 
df_tiempo_por_paso = (df_web_tipo_cliente.groupby(['Variation', 'process_step'])['duration_seconds'].median().reset_index())


# Renombrar la columna 
df_tiempo_por_paso = df_tiempo_por_paso.rename(columns={'duration_seconds': 'median_duration_seconds'})

# Para hacer este cálculo decidimos usar la mediana, ya que al usar la media el tiempo del proceso en cada paso era demasiado largo, 
# lo que reflejaba outliers que pueden presentarse por diferentes causas,como que un cliente deje abierto el proceso por mucho tiempo 
# sin estar trabajando en el.
# Con la mediana obtenemos una visión mucho más realista del tiempo dedicado por los usuarios.

In [ ]:
# Pivotar el tiempo_por_paso para tener las variaciones como columnas
df_duracion = df_tiempo_por_paso.pivot(index='process_step', columns='Variation', values='median_duration_seconds')
df_duracion

In [ ]:
# Calcular las diferencias 
df_duracion['diferencia_segundos'] = df_duracion['Control'] - df_duracion['Test']

# Mejora porcentual 
df_duracion['mejora_%'] = (((df_duracion['Control'] - df_duracion['Test']) / df_duracion['Control']) * 100).round(2)

# Reorganizar las columnas
df_diferencia_final = df_duracion.reset_index()

df_diferencia_final